In [ ]:
%reset -f
import time
import pickle
import gym
from scipy.interpolate import UnivariateSpline
from stable_baselines3 import DDPG
import matplotlib.pyplot as plt
from stable_baselines3.common.env_checker import check_env
import numpy as np
from tqdm import tqdm
import derivatives as d
import forward_helpers as fh
from chameleon import Chameleon

In [ ]:
n_elems = 100
steps = 1000
chameleon = Chameleon(E=100, n_elems=n_elems)

In [ ]:
??fh.update_disp

In [ ]:
# breaking_action = [0.42846936, 0.2765398, 0.9082336 ]

In [ ]:
# E = 1, 10 works E = 0.1, 100 breaks
active_stress = np.array(
    [
        0.42846936,
        0.47040876,
        0.53477369,
        0.62156414,
        0.7307801,
        0.86242159,
        1.0164886,
        1.19298113,
        1.39189918,
        1.61324275,
    ]
)

# np.save("breaking_stress", active_stress)

In [ ]:
fh.forward_simulate(chameleon, active_stress, sim_steps=steps)

In [ ]:
chameleon.pos_f

In [ ]:
chameleon.position_history

In [ ]:
x = chameleon.pos_0

In [ ]:
x

In [ ]:
chameleon.disp_current

In [ ]:
chameleon.position_history

# need to be able to compute the displacement for one step by hand and compare to what I get here. I want to use the first order method of enforcing the BC because that has a smooth simulation video but it may be less accurate than second order method which oscillates like crazy in the simulation. 

## Is this is too slow? In order to do RL I may need to be able to simulate much faster than this. (Jax?)

## There seems to be an issue when the stress is not monotonically increasing (actually more complex than this). The second element often ends up behind the first and as a result the sim crashes. I am not exactly sure why it is happening but it is clearly related to the boundary condition being imposed on the first element. If I then try to make sure the second element is always beyond the first, the third element goes behind the second. I think the issue is related to the stress on the first element causing a high force since the 0->1 element gets stretched a lot and it may be less than the stress from the 1->2 element. If the active stress is not enough to overcome that, then the second element would move to the left and maybe end up behind the first element. 

Actually i don't buy that argument because then a uniform stress might also cause this issue. In that case we would have a large stress from the stretching 0->1 element and if the active stress wasn't enough to overcome it we could end up with the same issue. But whenever I run uniform simulations I don't see that problem (no matter the magnitude of the active stress).

One way to debug is to save the NET force (and component forces maybe) on each element at each point in time and to see what happens. 

# I think its the Derivative of $\sigma$ that's important here.

## Why do the elements come in after the first time step? 

Then after the first time step is over they move out as expected. Strange. It's only the element on the end and it comes from satisfying the boundary condition.

# some hacky code to store the position history for later visualization.


In [ ]:
history = []
for i in range(len(chameleon.position_history)):
    x_p = chameleon.position_history[i]
    y = np.zeros(chameleon.n_elems)
    ar = np.vstack((x_p, y)).T
    history.append(ar)

with open("history.txt", "wb") as fp:
    pickle.dump(history, fp)